In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import re 
from sklearn.model_selection import train_test_split

In [9]:
import spacy
from spacy.tokens import Doc
nlp = spacy.load("en_core_web_lg")

In [12]:
cmu_coreNLP = pd.read_csv("cmu_coreNLP.tsv", sep='\t')

In [13]:
display(cmu_coreNLP)

,WikiMovieID,FreebaseMovieID,MovieName,ReleaseDate,BORevenue,Languages,Countries,MovieGenre,ReleaseYear,CharacterName,...,Attributes,Mentions,MainCharacter,Attributes_nlp,AgentVerbs_nlp,PatientVerbs_nlp,Attributes_nlp_vector,AgentVerbs_nlp_vector,PatientVerbs_nlp_vector,Character_nlp
0,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,{'/m/02h40lc': 'English Language'},{'/m/0hzlz': 'South Africa'},"{'/m/0hqxf': 'Family Film', '/m/01hmnh': 'Fant...",2002.0,Morgana,...,[],2,False,NaN,see,NaN,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 3.359 3.0831 0.5473 -1.3214 -2.42...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 1.1196667 1.0277001 0.18243332 -0.440466...
1,6631279,/m/0gffwj,Little city,1997-04-04,NaN,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/06cvj': 'Romantic comedy', '/m/0hj3n0w': ...",1997.0,Rebecca,...,[],3,False,NaN,begins,ruin,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 4.0952 3.1 -0.48909 -1.5118 5.00...,[-3.0634e-01 -1.3450e+00 -2.4759e+00 -4.5135e+...,[ 1.26295340e+00 5.84999979e-01 -9.88329947e-...
2,6631279,/m/0gffwj,Little city,1997-04-04,NaN,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/06cvj': 'Romantic comedy', '/m/0hj3n0w': ...",1997.0,Nina,...,[],4,False,NaN,threatens,move,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.0652e+00 2.5128e-01 -1.1949e+00 1.1009e+...,[ 0.29981 8.0302 -5.4315 -0.38133 ...,[ 7.88336694e-01 2.76049328e+00 -2.20880008e+...
3,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,{'/m/02h40lc': 'English Language'},{'/m/07ssc': 'United Kingdom'},"{'/m/04xvh5': 'Costume drama', '/m/082gq': 'Wa...",1989.0,Henry V,...,[],2,True,NaN,NaN,made,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ -9.1307 -3.3238 5.5397 -6.6699 ...,[-3.0435667 -1.1079334 1.8465667 -2.2233 ...
4,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/0hj3myq': ""Children's/Family"", '/m/04t36'...",1964.0,George Banks,...,[],4,False,NaN,have,tells,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ -2.2659 4.1936 -6.7259 1.0432 ...,[ 4.4343 -1.1702 2.8177 -2.3501 1.32...,[ 0.7228 1.0078001 -1.3027334 -0.435633...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35545,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,"{'/m/02bjrlw': 'Italian Language', '/m/02h40lc...","{'/m/0f8l9c': 'France', '/m/09c7w0': 'United S...","{'/m/0gf28': 'Parody', '/m/09n5t_': 'Americana...",2000.0,Bob Barrenger,...,[],1,False,NaN,NaN,NaN,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
35546,15394941,/m/03m6zh4,Gopi Kishan,1994-12-02,NaN,{'/m/03k50': 'Hindi Language'},{'/m/03rk0': 'India'},"{'/m/07s9rl0': 'Drama', '/m/02kdv5l': 'Action'...",1994.0,Sawant,...,{'head'},16,False,head,caught,infuriated engage,[ 3.6923e+00 1.8720e+00 -4.0144e+00 5.2259e+...,[-1.8047 1.0036 -2.9816 -1.4792 1.30...,[-0.8085 -0.250095 0.6814 -0.21768 ...,[ 0.35970005 0.8751683 -2.1048667 1.176340...
35547,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,{'/m/03_9r': 'Japanese Language'},{'/m/03_3d': 'Japan'},"{'/m/06n90': 'Science Fiction', '/m/0gw5n2f': ...",1992.0,Hibiki Kanzaki,...,[],3,True,NaN,end,NaN,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ -5.9623 2.9037 -2.1551 3.5432 ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.9874333 0.96790004 -0.7183667 1.181066...
35548,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,{'/m/03_9r': 'Japanese Language'},{'/m/03_3d': 'Japan'},"{'/m/06n90': 'Science Fiction', '/m/0gw5n2f': ...",1992.0,Hibiki Kanzaki,...,[],3,True,NaN,end,NaN,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ -5.9623 2.9037 -2.1551 3.5432 ...,

In [17]:
cmu_coreNLP.Attributes_nlp_vector = cmu_coreNLP.Attributes_nlp_vector.apply(lambda x : np.array([float(match.group()) for match in re.finditer(r'\d+\.\d+', x)]))

In [19]:
cmu_coreNLP.AgentVerbs_nlp_vector = cmu_coreNLP.AgentVerbs_nlp_vector.apply(lambda x : np.array([float(match.group()) for match in re.finditer(r'\d+\.\d+', x)]))
cmu_coreNLP.PatientVerbs_nlp_vector = cmu_coreNLP.PatientVerbs_nlp_vector.apply(lambda x : np.array([float(match.group()) for match in re.finditer(r'\d+\.\d+', x)]))

In [46]:
ylabels = pd.get_dummies(cmu_coreNLP['Gender'], dtype=int)
ylabels = ylabels['F']

big_X_attributes = pd.DataFrame([arr for arr in cmu_coreNLP['Attributes_nlp_vector']])
big_X_agent = pd.DataFrame([arr for arr in cmu_coreNLP['AgentVerbs_nlp_vector']])
big_X_patient = pd.DataFrame([arr for arr in cmu_coreNLP['PatientVerbs_nlp_vector']])

big_X = pd.concat([big_X_attributes, big_X_agent], axis=1)
big_X = pd.concat([big_X, big_X_patient], axis=1)

big_X["labels"] = ylabels

big_X = big_X.dropna()
display(big_X)

ylabels = big_X["labels"].to_numpy()
big_X = big_X.drop(columns="labels")
display(big_X)

X = big_X.to_numpy()


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,labels
11,2.173833,3.047933,8.43400,6.994000,1.03740,3.461667,3.144033,3.830333,6.486333,6.508333,...,3.520286,1.951429,1.442286,2.927714,4.534143,9.888286,9.036714,3.581143,1.590000,1
13,1.316600,0.480740,0.77055,0.074920,0.56535,0.618900,1.928600,0.631750,1.353100,0.093435,...,0.403633,0.550033,1.042567,0.284337,0.876200,0.089833,0.768100,0.611067,0.176223,0
17,4.309000,6.039600,2.94050,2.081600,9.92760,1.419800,3.525000,5.682600,5.370100,3.339300,...,2.082900,2.417200,3.613700,1.018800,1.444200,1.384400,0.584690,2.368800,0.879780,0
25,3.734300,1.824500,2.38190,3.648900,1.08150,0.915280,0.053149,0.822380,2.515700,2.142800,...,0.127828,0.424120,0.592100,0.492860,0.776120,1.015300,0.982180,0.949480,0.436600,1
30,1.177700,3.768400,9.85950,3.330300,1.40310,6.450800,7.691000,8.660700,6.933000,1.043000,...,1.544800,7.574200,3.412000,1.026400,3.845300,3.389400,1.877700,1.648300,1.583800,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35524,1.282900,2.130600,2.50920,0.857360,2.28330,1.925600,2.932500,2.297600,0.657110,3.776800,...,2.385950,7.747000,3.099300,7.967000,1.537600,3.421500,1.182600,1.295350,1.950900,1
35525,0.478095,1.210250,1.95420,0.239765,1.87145,1.858700,1.086650,1.494500,0.116340,1.900400,...,0.580300,1.198433,0.033413,0.174500,1.164067,0.533800,1.741567,0.523533,0.472967,1
35528,5.973100,3.861000,8.21810,3.281100,5.20590,1.962800,1.965700,2.842000,1.205200,3.983100,...,0.411813,0.459875,0.309075,0.152375,0.082295,0.035006,0.019549,0.087987,0.067560,0
35533,0.682210,2.141700,3.70430,2.260100,0.65744,0.255800,0.905340,4.216800,6.116500,5.482500,...,0.068247,0.182653,0.131193,0.231620,1.353800,2.338900,1.855500,0.038110,0.594200,1


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
11,2.173833,3.047933,8.43400,6.994000,1.03740,3.461667,3.144033,3.830333,6.486333,6.508333,...,7.779428,3.520286,1.951429,1.442286,2.927714,4.534143,9.888286,9.036714,3.581143,1.590000
13,1.316600,0.480740,0.77055,0.074920,0.56535,0.618900,1.928600,0.631750,1.353100,0.093435,...,1.271267,0.403633,0.550033,1.042567,0.284337,0.876200,0.089833,0.768100,0.611067,0.176223
17,4.309000,6.039600,2.94050,2.081600,9.92760,1.419800,3.525000,5.682600,5.370100,3.339300,...,2.611400,2.082900,2.417200,3.613700,1.018800,1.444200,1.384400,0.584690,2.368800,0.879780
25,3.734300,1.824500,2.38190,3.648900,1.08150,0.915280,0.053149,0.822380,2.515700,2.142800,...,0.673320,0.127828,0.424120,0.592100,0.492860,0.776120,1.015300,0.982180,0.949480,0.436600
30,1.177700,3.768400,9.85950,3.330300,1.40310,6.450800,7.691000,8.660700,6.933000,1.043000,...,1.972200,1.544800,7.574200,3.412000,1.026400,3.845300,3.389400,1.877700,1.648300,1.583800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35524,1.282900,2.130600,2.50920,0.857360,2.28330,1.925600,2.932500,2.297600,0.657110,3.776800,...,1.451650,2.385950,7.747000,3.099300,7.967000,1.537600,3.421500,1.182600,1.295350,1.950900
35525,0.478095,1.210250,1.95420,0.239765,1.87145,1.858700,1.086650,1.494500,0.116340,1.900400,...,1.544633,0.580300,1.198433,0.033413,0.174500,1.164067,0.533800,1.741567,0.523533,0.472967
35528,5.973100,3.861000,8.21810,3.281100,5.20590,1.962800,1.965700,2.842000,1.205200,3.983100,...,0.538212,0.411813,0.459875,0.309075,0.152375,0.082295,0.035006,0.019549,0.087987,0.067560
35533,0.682210,2.141700,3.70430,2.260100,0.65744,0.255800,0.905340,4.216800,6.116500,5.482500,...,0.369133,0.068247,0.182653,0.131193,0.231620,1.353800,2.338900,1.855500,0.038110,0.594200


In [47]:
X.shape

(8961, 900)

In [48]:
# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, ylabels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # Assuming a binary classification problem
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

In [61]:
class GenderClassifier(nn.Module):
    def __init__(self, input_size, layer_sizes, output_size):
        super(GenderClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, layer_sizes[0])
        self.relu = nn.LeakyReLU()
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.relu = nn.LeakyReLU()
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.relu = nn.LeakyReLU()
        self.fc4 = nn.Linear(layer_sizes[2], output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# Set the dimensions
input_size = 900  # Number of features in the word vectors
layer_sizes = [64, 64, 64]  # Number of units in the hidden layer
output_size = 1  # Binary classification, so one output node with a sigmoid activation

# Instantiate the model
model = GenderClassifier(input_size, layer_sizes, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the number of epochs
epochs = 30

# Training loop
for epoch in range(epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Evaluation on the validation set
    model.eval()
    val_outputs = model(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)

    # Print training and validation loss for each epoch
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')

# Test the model on the test set
model.eval()
test_outputs = model(X_test_tensor)
test_loss = criterion(test_outputs, y_test_tensor)
print(f'Test Loss: {test_loss.item()}')

# Evaluation on the test set
test_outputs = model(X_test_tensor)
test_loss = criterion(test_outputs, y_test_tensor)
test_preds = (test_outputs > 0.5).float()
test_accuracy = (test_preds == y_test_tensor).float().mean()

print(test_accuracy)

Epoch 1/30, Loss: 0.7105949521064758, Val Loss: 0.6392413377761841
Epoch 2/30, Loss: 0.6393161416053772, Val Loss: 0.6514368057250977
Epoch 3/30, Loss: 0.6481198668479919, Val Loss: 0.650009274482727
Epoch 4/30, Loss: 0.6471542119979858, Val Loss: 0.6353087425231934
Epoch 5/30, Loss: 0.6339169144630432, Val Loss: 0.6304268836975098
Epoch 6/30, Loss: 0.6304821968078613, Val Loss: 0.6329901814460754
Epoch 7/30, Loss: 0.6334574818611145, Val Loss: 0.6325278282165527
Epoch 8/30, Loss: 0.632800281047821, Val Loss: 0.6305525898933411
Epoch 9/30, Loss: 0.6304486393928528, Val Loss: 0.6314917802810669
Epoch 10/30, Loss: 0.6307513117790222, Val Loss: 0.6329489946365356
Epoch 11/30, Loss: 0.6316972374916077, Val Loss: 0.6322911381721497
Epoch 12/30, Loss: 0.6307969093322754, Val Loss: 0.6308327913284302
Epoch 13/30, Loss: 0.6292622089385986, Val Loss: 0.6302403211593628
Epoch 14/30, Loss: 0.6285993456840515, Val Loss: 0.6305258870124817
Epoch 15/30, Loss: 0.628868818283081, Val Loss: 0.630692660